In [1]:
import numpy as np
import pandas as pd
house=pd.read_csv("_All_Cities_Cleaned.csv")

In [2]:
house.head()

,seller_type,bedroom,layout_type,property_type,locality,price,area,furnish_type,bathroom,city
0,OWNER,2.0,BHK,Apartment,Bodakdev,20000.0,1450.0,Furnished,2.0,Ahmedabad
1,OWNER,1.0,RK,Studio Apartment,CG Road,7350.0,210.0,Semi-Furnished,1.0,Ahmedabad
2,OWNER,3.0,BHK,Apartment,Jodhpur,22000.0,1900.0,Unfurnished,3.0,Ahmedabad
3,OWNER,2.0,BHK,Independent House,Sanand,13000.0,1285.0,Semi-Furnished,2.0,Ahmedabad
4,OWNER,2.0,BHK,Independent House,Navrangpura,18000.0,1600.0,Furnished,2.0,Ahmedabad


In [6]:
house['area'].unique()

array([1450.,  210., 1900., ..., 3575., 1721.,  258.])

In [4]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193011 entries, 0 to 193010
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   seller_type    193011 non-null  object 
 1   bedroom        193011 non-null  float64
 2   layout_type    193011 non-null  object 
 3   property_type  193011 non-null  object 
 4   locality       193011 non-null  object 
 5   price          193011 non-null  float64
 6   area           193011 non-null  float64
 7   furnish_type   193011 non-null  object 
 8   bathroom       193011 non-null  float64
 9   city           193011 non-null  object 
dtypes: float64(4), object(6)
memory usage: 14.7+ MB


In [5]:
house.describe()

,bedroom,price,area,bathroom
count,193011.000000,1.930110e+05,193011.000000,193011.000000
mean,2.081679,4.433654e+04,1264.604468,2.059883
std,0.956901,9.195199e+04,1043.725561,0.934805
min,1.000000,1.200000e+03,3.000000,1.000000
25%,1.000000,1.300000e+04,650.000000,1.000000
50%,2.000000,2.100000e+04,1000.000000,2.000000
75%,3.000000,3.600000e+04,1440.000000,3.000000
max,15.000000,5.885000e+06,19800.000000,19.000000


In [26]:
house.corr()

,bedroom,price,area,bathroom
bedroom,1.000000,0.526540,0.787946,0.875798
price,0.526540,1.000000,0.756773,0.534605
area,0.787946,0.756773,1.000000,0.778283
bathroom,0.875798,0.534605,0.778283,1.000000


In [6]:
X = house.drop(columns='price')
y=house['price']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [9]:
ohe  = OneHotEncoder()
ohe.fit(X[['seller_type','layout_type','property_type','locality','furnish_type','city']])

OneHotEncoder()

In [10]:
column_trans = make_column_transformer((OneHotEncoder(categories= ohe.categories_),['seller_type','layout_type','property_type','locality','furnish_type','city']),
                                       remainder = 'passthrough')

In [11]:
lr = LinearRegression()

In [12]:
pipe = make_pipeline(column_trans, lr)

In [13]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['AGENT', 'BUILDER', 'OWNER'], dtype=object),
                                                                            array(['BHK', 'RK'], dtype=object),
                                                                            array(['Apartment', 'Independent Floor', 'Independent House', 'Penthouse',
       'Studio Apartment', 'Villa'], dtype=object),
                                                                            array(['1 Sector Number 3 R...
       '10 Sector Number 6 Road', ..., 'wakad', 'worli sea Fase',
       'yogi nagar'], dtype=object),
                                                                            array(['Furnished', 'Semi-Furnished', 'Unfurnished'], dtype=object),
                                     

In [14]:
y_pred = pipe.predict(X_test)

In [15]:
r2_score(y_test,y_pred)

0.7903920885286109

In [21]:
scores=[]
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(column_trans, lr)
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [22]:
scores[np.argmax(scores)]


0.8046934306804441

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe = make_pipeline(column_trans, lr)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8046934306804441

In [24]:
import pickle

In [25]:
pickle.dump(pipe,open('Self_LinearRegressionModel.pkl','wb'))